<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/python_docx%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E4%B8%A1%E5%81%B4%E3%83%AB%E3%83%93.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# python-docxを用いた両側ルビ



In [24]:
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
from bs4 import BeautifulSoup
import docx
from docx.oxml import OxmlElement

def convert(document, soup, parent):
    '''
    変換メソッド
    '''

    name = soup.name

    if name == None:
        r = OxmlElement('w:r')
        e = OxmlElement('w:instrText')

        text = str(soup).strip()

        if text == "":
            return
        
        e.text = text
        r.append(e)
        parent.append(r)

    elif name == "ruby":
        rb = soup.findChildren("rb", recursive=False)[0]
        rt = soup.findChildren("rt", recursive=False)[0]

        place = rt.get("place")
        position = "up" if place in ["right", "above"] else "do"

        # w:r
        r = OxmlElement('w:r')
        e = OxmlElement('w:fldChar')
        r.append(e)
        e.set(docx.oxml.shared.qn('w:fldCharType'), 'begin')
        parent.append(r)

        # w:r
        r = OxmlElement('w:r')
        e = OxmlElement('w:instrText')
        e.text = f'EQ \\o(\\s\\{position} 10('
        r.append(e)
        parent.append(r)

        # w:r
        r = OxmlElement('w:r')

        e = OxmlElement('w:rPr')
        e2 = OxmlElement('w:sz')
        e2.set(docx.oxml.shared.qn('w:val'), str(11))
        e.append(e2)
        r.append(e)

        e = OxmlElement('w:instrText')
        e.text = rt.text.strip()
        r.append(e)
        parent.append(r)

        # w:r
        r = OxmlElement('w:r')
        e = OxmlElement('w:instrText')
        e.text = "),"
        r.append(e)
        parent.append(r)

        es = rb.children
        for e in es:
            convert(document, e, parent)

        # w:r
        r = OxmlElement('w:r')
        e = OxmlElement('w:instrText')
        e.text = ')'
        r.append(e)
        parent.append(r)

        # w:r
        r = OxmlElement('w:r')
        e = OxmlElement('w:fldChar')
        e.set(docx.oxml.shared.qn('w:fldCharType'), 'end')
        r.append(e)
        parent.append(r)

    elif name == "p":
        p = document.add_paragraph()
        es = soup.children
        for e in es:
            convert(document, e, p._p)

    else:
        es = soup.children
        for e in es:
            convert(document, e, parent)

In [26]:
html = '''<body>
<p>
 私は
 <ruby>
  <rb>
   <ruby>
    <rb>打</rb>
    <rt place="right">ダ</rt>
   </ruby>
   <ruby>
    <rb>球</rb>
    <rt place="right">キウ</rt>
   </ruby>
     場
  </rb>
  <rt place="left">ビリヤード</rt>
 </ruby>
 に行きました。
</p><p>
 <ruby>
  <rb>入学試験</rb>
  <rt place="above">にゅうがくしけん</rt>
 </ruby>
 があります。
</p>
</body>'''

soup = BeautifulSoup(html, 'html.parser')
document = docx.Document()

convert(document, soup, None)
document.save('test.docx')